# GPC-CS Template

This notebook is designed to analyze and visualize  the effects of a specific genetic variant on various phenotypic features. You need to fill in the following sections with actual code and data to perform the analysis.
> Tip: Change all "[GeneName]" placeholders to the actual gene name you are analyzing.

In [ ]:
%%sql


# [GeneName] and Associated Syndrome

<!-- Provide a brief overview of the syndrome and its genetic basis. Replace [Gene Name] and [Syndrome Name] with the specific gene and syndrome you are studying. -->

[[Syndrome Name]](link_to_syndrome) is an [type of disease, e.g. autosomal dominant] disease characterized by [describe main features of the syndrome]. The syndrome is caused by pathogenic variants in the [[GeneName]](link_to_gene) gene.

<!-- Summarize key findings from previous studies on genotype-phenotype correlations in the syndrome. -->

One/Two/Several previous publications have reported candidate genotype-phenotype correlations in [Syndrome Name].

<!-- Cite specific studies and their findings. Replace with relevant studies and findings for the specific gene and syndrome. -->

[Author et al. (Year)](link_to_study) stated that:

> [Quote relevant finding from the study]

[Author et al. (Year)](link_to_study) stated that:

> [Quote relevant finding from the study]

[Author et al. (Year)](link_to_study) report:

> [Quote relevant finding from the study]

<!-- TODO: Add instructions for finding more comprehensive citations and explain the next steps in your analysis. -->

TODO -- find a more comprehensive collection of citations for [GeneName]. At the end of the introduction for a gene, we would write something like this:

> Example: Therefore, we tested missense vs other variants (inspection of the distribution of variants showed that the other variant categories in our dataset are [list other variant categories], all of which we deemed to be null variants for the purposes of this analysis). We also tested the two most common missense variants ([DETAILS]) for correlations.


## Imports 

In [20]:
import gpsea
from gpsea.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from gpsea.analysis.predicate import PatientCategories
from gpsea.preprocessing import load_phenopacket_folder
from gpsea.preprocessing import configure_caching_cohort_creator
from gpsea.model import FeatureType, VariantEffect
from gpsea.view import CohortViewable
from gpsea.preprocessing import UniprotProteinMetadataService
from gpsea.model.genome import GRCh38
from gpsea.preprocessing import VVTranscriptCoordinateService
from gpsea.view import ProteinVisualizable, ProteinVisualizer, ProteinViewable
import hpotk
from IPython.display import display, HTML

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2023-10-09')
print(f'Loaded HPO v{hpo.version}')
print(f"Using gpsea version {gpsea.__version__}")


ModuleNotFoundError: No module named 'genophenocorr'

## Loading Phenopackets & Gene Variant Data
The prefered transcript can be found by searching on the gene symbol in [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/). By entering the accession number in [NCBI Nucleotide](https://www.ncbi.nlm.nih.gov/nuccore/?), you can find the corresponding protein accession number.

In [ ]:
[GeneName]_MANE_transcript = 'NM_...'
[GeneName]_protein_id = "NP_..."
cohort_creator = configure_caching_cohort_creator(hpo, timeout=20)
phenopacket_input_folder = "../../../../GIT/phenopacket-store/notebooks/[GeneName]/phenopackets/"
cohort = load_phenopacket_folder(pp_directory=phenopacket_input_folder, cohort_creator=cohort_creator)

## Define Configuration & Run Analysis

In [ ]:
cv = CohortViewable(hpo=hpo)
html = cv.process(cohort=cohort, transcript_id=[GeneName]_MANE_transcript)
display(HTML(html))

In [ ]:
pms = UniprotProteinMetadataService()
protein_meta = pms.annotate([GeneName]_protein_id)
# TODO: Check Genome Build
txc_service = VVTranscriptCoordinateService(genome_build=GRCh38)
tx_coordinates = txc_service.fetch([GeneName]_MANE_transcript)
pvis = ProteinVisualizable(tx_coordinates=tx_coordinates, protein_meta=protein_meta, cohort=cohort)

In [ ]:
viewer = ProteinViewable()
html_prot = viewer.process(cohort, pvis)
display(HTML(html_prot))

In [ ]:
drawer = ProteinVisualizer()
drawer.draw_fig(pvis=pvis);

## Correlation Analysis

In [ ]:
analysis_config = CohortAnalysisConfiguration()
analysis_config.missing_implies_excluded = False
# TODO: Check HPO_observed_frequency 
analysis_config.heuristic_strategy(threshold_HPO_observed_frequency=0.2)
analysis = configure_cohort_analysis(cohort, hpo, config=analysis_config)

In [ ]:
from gpsea.model import FeatureType

# TODO: Check compare_by_variant_effect parameters and/or variables 
frameshift = analysis.compare_by_variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id=[GeneName]_MANE_transcript)
frameshift.summarize(hpo, PatientCategories.YES)

In [ ]:
# TODO: Check compare_by_variant_key parameters and/or variables
feature = analysis.compare_by_variant_key(variant_key="12_114385521_114385521_C_T")
feature.summarize(hpo, PatientCategories.YES)

In [ ]:
# TODO: Check compare_by_variant_key parameters and/or variables
feature = analysis.compare_by_variant_key(variant_key="12_114401830_114401830_C_T")
feature.summarize(hpo, PatientCategories.YES)